In [1]:
import cudaq

# Initialize MPI
cudaq.mpi.initialize()

# Get the rank of the current process
rank = cudaq.mpi.rank()

print(f"Process rank: {rank}")

# Finalize MPI
cudaq.mpi.finalize()


RuntimeError: No MPI support can be found when attempted to use cudaq::mpi APIs. Please refer to the documentation for instructions to activate MPI support.

In [3]:
import subprocess
import cudaq

qubit_count = 32
def get_gpu_memory_usage():
    result = subprocess.run(['nvidia-smi', '--query-gpu=memory.used,memory.free', '--format=csv,noheader,nounits'],
                            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode != 0:
        raise RuntimeError(f"Error querying GPU memory: {result.stderr}")
    memory_used, memory_free = map(int, result.stdout.strip().split(', '))
    return memory_used, memory_free

def run_quantum_circuit(shots):
    try:
        # Initialize your quantum circuit here
        @cudaq.kernel
        def kernel(qubit_count: int):
            qvector = cudaq.qvector(qubit_count)
            h(qvector)
            for qubit in range(qubit_count - 1):
                x.ctrl(qvector[qubit], qvector[qubit + 1])
            mz(qvector)
        result = cudaq.sample(kernel, qubit_count, shots_count=shots)
        return True
    except RuntimeError as e:
        print(f"Runtime error with {shots} shots: {e}")
        return False
    except Exception as e:
        print(f"Unexpected error with {shots} shots: {e}")
        return False

def binary_search_max_shots(low, high):
    while low <= high:
        mid = (low + high) // 2
        memory_used_before, memory_free_before = get_gpu_memory_usage()
        success = run_quantum_circuit(mid)
        memory_used_after, memory_free_after = get_gpu_memory_usage()

        print(f"Shots: {mid}, Success: {success}, Memory used: {memory_used_after - memory_used_before} MB, Memory free: {memory_free_after} MB")
        
        if success:
            low = mid + 1
        else:
            high = mid - 1
    return high

# Initial range for binary search
low_shots = 1000
high_shots = 1050000

max_shots = binary_search_max_shots(low_shots, high_shots)
print(f"Maximum shots without segmentation fault: {max_shots}")


Shots: 500500, Success: True, Memory used: 428 MB, Memory free: 39492 MB
Shots: 750250, Success: True, Memory used: 0 MB, Memory free: 39492 MB
Shots: 875125, Success: True, Memory used: 0 MB, Memory free: 39492 MB
Shots: 937563, Success: True, Memory used: 0 MB, Memory free: 39492 MB
Shots: 968782, Success: True, Memory used: 0 MB, Memory free: 39492 MB
Shots: 984391, Success: True, Memory used: 0 MB, Memory free: 39492 MB
Shots: 992196, Success: True, Memory used: 0 MB, Memory free: 39492 MB
Shots: 996098, Success: True, Memory used: 0 MB, Memory free: 39492 MB
Shots: 998049, Success: True, Memory used: 0 MB, Memory free: 39492 MB
Shots: 999025, Success: True, Memory used: 0 MB, Memory free: 39492 MB
Shots: 999513, Success: True, Memory used: 0 MB, Memory free: 39492 MB
Shots: 999757, Success: True, Memory used: 0 MB, Memory free: 39492 MB
Shots: 999879, Success: True, Memory used: 0 MB, Memory free: 39492 MB
Shots: 999940, Success: True, Memory used: 0 MB, Memory free: 39492 MB
Shot